In [2]:
import pandas as pd
pd.reset_option('display.float_format')
df_ffca= pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\FFCA.xlsx")
df_qv = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\QV.xlsx")
df_stock = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\stock.xlsx")
df_stock.drop([0])
df_movement = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\Movement.xlsx")
df_inventory = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\inventory.xlsx")

pd.options.display.float_format = '{:.2f}'.format

In [3]:
df_vessel = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\vessel.xlsx")

In [15]:
#column filter
cols_ffca = ['Purchase Assignment ID', 'Commodity', 'Inventory ID','Counterparty of Purchase', 'QTY', \
    'Purchase Settled Amount', 'Collateral Doc Type', 'Financing Bank facility', 'Purchase Settled Amount', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value']

cols_qv = ['Assignment ID', 'Group Company', 'Trade ID', 'Quota ID', 'Traffic Operator', 'Collateral Doc Date', 'Transport Mode']

cols_movement = ['ID', 'LOAD DATE','LOAD', 'DESTINATION', 'DESTINATION DATE']

cols_inventory = ['INVENTORY ID', 'COLLATERAL DOCUMENT DATE', 'PURCHASE ASSIGNMENT', 'MOVEMENT ID','RECEIPT INCOTERMS', 'DRY WEIGHT', 'WET WEIGHT']

cols_stock = ['Inventory Id','Origin Country']

cols_vessel = ['TRANSPORT', 'INVENTORY ID']


#转换数据类型

df_stock_f1 = pd.DataFrame(df_stock, columns=cols_stock)
df_vessel_f1 = pd.DataFrame(df_vessel, columns=cols_vessel)
df_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca)
df_qv_f1 = pd.DataFrame(df_qv, columns=cols_qv)
df_movement_f1 = pd.DataFrame(df_movement, columns=cols_movement)
df_inventory_f1 = pd.DataFrame(df_inventory, columns=cols_inventory)
df_movement_f1['LOAD'] = df_movement_f1['LOAD'].astype(str)

#deliver detail
df_movement_f1.loc[:, 'Load Country'] = df_movement_f1['LOAD'].apply(lambda st: st[st.rfind("(")+1:st.rfind(")")])
df_movement_f1.loc[:, 'Delivery Country'] = df_movement_f1['DESTINATION'].apply(lambda st: st[st.rfind("(")+1:st.find(")")])


In [16]:
def blend(a):
    return a == "Blend"

df_blend_inventory = df_inventory_f1.loc[df_inventory_f1['PURCHASE ASSIGNMENT'].apply(blend)]

row_list = df_inventory_f1[df_inventory_f1['PURCHASE ASSIGNMENT'] == 'Blend'].index.tolist() # 获得含有该值的行的行号
df_nblend_inventory = df_inventory_f1.drop(row_list)

ffca_qv = pd.merge(df_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'inner')
bmovement_inventory = pd.merge(df_movement_f1, df_blend_inventory, left_on=["ID"], right_on=["MOVEMENT ID"], how='outer')
bMIS = pd.merge(bmovement_inventory, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
bfinal_BB1 = pd.merge(ffca_qv, bMIS, left_on=['Inventory ID'], right_on=['INVENTORY ID'])
bfinal_BB = pd.merge(bfinal_BB1, df_vessel_f1, on=['INVENTORY ID'])



def function(a, b, c):
    c * (1 + (a-b)/a)

bfinal_BB['Purchased Wet Weight']  = bfinal_BB.apply(lambda x: function(x['WET WEIGHT'], x['DRY WEIGHT'], x['QTY']), axis = 1)

bfinal_BB = bfinal_BB.drop(columns = ['WET WEIGHT', 'DRY WEIGHT'])

bfinal_BB.columns
#改名
bfinal_BB.columns = ['Purchase Assignment Reference', 'Purchase Commodity', 'Purchase Inventory ID', 'Purchase Contractual Counterparty', \
    'Purchase Dry Weight', 'Purchase Settlement Valuation(USD)', 'Collateral Document Type', 'Financing Bank facility', 'Purchase Settled Amount', 'Authorized Paid Quantity', \
        'Financed Quantity', 'In Transit Current Value(USD)', 'Assignment ID', 'Purchase Group Company Code', 'Purchase Trade ID', \
            'Purchase Quota Titan ID', 'Purchase Traffic Operator', 'Collateral Document Date', 'Purchase Transport Mode', 'ID', \
                 'Load Date', 'Load Location', 'Delivery Location', 'Expected Delivery Date', 'Load Country', 'Destination Country', \
                    'INVENTORY ID', 'COLLATERAL DOCUMENT DATE', 'PURCHASE ASSIGNMENT', 'MOVEMENT ID', 'Purchase Incoterm', 'Inventory Id','Purchase Material Origin Country','Vessel Name', 'Purchase Wet Weight']


#新增固定列
bfinal_BB['Collateral Reference'] = None
bfinal_BB['Purchase Dry Weight UOM'] = 'MT'
bfinal_BB['Purchase Wet Weight UOM'] = 'MT'
bfinal_BB['Receipt In Progress'] = 'N'
bfinal_BB['Delivery In Progress'] = 'N'
# bfinal_BB['Receipt In Progress'] = bfinal_BB.apply(lambda x: function(x['Receipt In Progress Qty (Wet)'], x['Inventory Quantity(Wet)']), axis = 1)
# bfinal_BB['Delivery In Progress'] = bfinal_BB.apply(lambda x: function(x['Delivery In Progress Qty'], x['Inventory Quantity(Wet)']), axis = 1)
bfinal_BB[['Authorized Paid Quantity', 'Financed Quantity']] = bfinal_BB[['Authorized Paid Quantity', 'Financed Quantity']].astype('float')
bfinal_BB.loc[:,"Financing Ratio"] = bfinal_BB["Authorized Paid Quantity"]/bfinal_BB["Financed Quantity"]

#复制列
bfinal_BB.loc[:, 'Eligible Dry Weight'] = bfinal_BB['Purchase Dry Weight']
bfinal_BB.loc[:, 'Eligible Wet Weight'] = bfinal_BB['Purchase Wet Weight']
bfinal_BB.loc[:, 'BL Date'] = bfinal_BB['COLLATERAL DOCUMENT DATE']


#删除不要的列
bfinal_BB.drop(columns=['Financing Bank facility', 'Purchase Settled Amount', 'Assignment ID', 'ID', \
    'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Collateral Document Date', 'MOVEMENT ID'])

#修改顺序
order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Purchase Inventory ID', \
    'Purchase Traffic Operator', 'Purchase Contractual Counterparty', 'Collateral Document Type', 'Collateral Reference', 'COLLATERAL DOCUMENT DATE', \
        'Purchase Commodity', 'Purchase Material Origin Country', 'Purchase Incoterm', 'Purchase Transport Mode', 'Vessel Name', 'Load Date', \
            'Load Location', 'Load Country', 'BL Date', 'Expected Delivery Date', 'Delivery Location', 'Destination Country', 'Purchase Dry Weight', \
                'Purchase Dry Weight UOM', 'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Purchase Settlement Valuation(USD)',\
                    'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight', 'Authorized Paid Quantity', \
                        'Financed Quantity','Financing Ratio', 'In Transit Current Value(USD)']
bfinal_BB_test = bfinal_BB[order]

In [19]:
movement_inventory = pd.merge(df_movement_f1, df_inventory_f1, left_on=["ID"], right_on=["MOVEMENT ID"], how='outer')
MIS = pd.merge(movement_inventory, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
final_BB1 = pd.merge(ffca_qv, MIS, left_on=['Inventory ID'], right_on=['INVENTORY ID'])
final_BB = pd.merge(final_BB1, df_vessel_f1, on=['INVENTORY ID'])


In [20]:
# #新增固定列
# def function(a,b):
#     if a == b:
#         return 'N'
#     elif a == 0:
#         return 'N'
#     else:
#         return 'Y'

#改名
final_BB.columns = ['Purchase Assignment Reference', 'Purchase Commodity', 'Purchase Inventory ID', 'Purchase Contractual Counterparty', \
    'QTY', 'Purchase Settlement Valuation(USD)', 'Collateral Document Type', 'Financing Bank facility', 'Purchase Settled Amount', 'Authorized Paid Quantity', \
        'Financed Quantity', 'In Transit Current Value(USD)', 'Assignment ID', 'Purchase Group Company Code', 'Purchase Trade ID', \
            'Purchase Quota Titan ID', 'Purchase Traffic Operator', 'Collateral Document Date', 'Purchase Transport Mode', 'ID', \
                 'Load Date', 'Load Location', 'Delivery Location', 'Expected Delivery Date', 'Load Country', 'Destination Country', \
                    'INVENTORY ID', 'COLLATERAL DOCUMENT DATE', 'PURCHASE ASSIGNMENT', 'MOVEMENT ID', 'Purchase Incoterm', 'Purchase Dry Weight', 'Purchase Wet Weight','Inventory Id','Purchase Material Origin Country','Vessel Name']


#新增固定列
final_BB['Collateral Reference'] = None
final_BB['Purchase Dry Weight UOM'] = 'MT'
final_BB['Purchase Wet Weight UOM'] = 'MT'
final_BB['Receipt In Progress'] = 'N'
final_BB['Delivery In Progress'] = 'N'
# final_BB['Receipt In Progress'] = final_BB.apply(lambda x: function(x['Receipt In Progress Qty (Wet)'], x['Inventory Quantity(Wet)']), axis = 1)
# final_BB['Delivery In Progress'] = final_BB.apply(lambda x: function(x['Delivery In Progress Qty'], x['Inventory Quantity(Wet)']), axis = 1)
final_BB[['Authorized Paid Quantity', 'Financed Quantity']] = final_BB[['Authorized Paid Quantity', 'Financed Quantity']].astype('float')
final_BB.loc[:,"Financing Ratio"] = final_BB["Authorized Paid Quantity"]/final_BB["Financed Quantity"]

#复制列
final_BB.loc[:, 'Eligible Dry Weight'] = final_BB['Purchase Dry Weight']
final_BB.loc[:, 'Eligible Wet Weight'] = final_BB['Purchase Wet Weight']
final_BB.loc[:, 'BL Date'] = final_BB['COLLATERAL DOCUMENT DATE']


#删除不要的列
final_BB.drop(columns=['QTY', 'Financing Bank facility', 'Purchase Settled Amount', 'Assignment ID', 'ID', \
    'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Collateral Document Date', 'MOVEMENT ID'])

#修改顺序
order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Purchase Inventory ID', \
    'Purchase Traffic Operator', 'Purchase Contractual Counterparty', 'Collateral Document Type', 'Collateral Reference', 'COLLATERAL DOCUMENT DATE', \
        'Purchase Commodity', 'Purchase Material Origin Country', 'Purchase Incoterm', 'Purchase Transport Mode', 'Vessel Name', 'Load Date', \
            'Load Location', 'Load Country', 'BL Date', 'Expected Delivery Date', 'Delivery Location', 'Destination Country', 'Purchase Dry Weight', \
                'Purchase Dry Weight UOM', 'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Purchase Settlement Valuation(USD)',\
                    'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight', 'Authorized Paid Quantity', \
                        'Financed Quantity','Financing Ratio', 'In Transit Current Value(USD)']
final_BB_test = final_BB[order]
final_BB_test

,Purchase Group Company Code,Purchase Trade ID,Purchase Quota Titan ID,Purchase Assignment Reference,Purchase Inventory ID,Purchase Traffic Operator,Purchase Contractual Counterparty,Collateral Document Type,Collateral Reference,COLLATERAL DOCUMENT DATE,...,Purchase Wet Weight UOM,Purchase Settlement Valuation(USD),Receipt In Progress,Delivery In Progress,Eligible Dry Weight,Eligible Wet Weight,Authorized Paid Quantity,Financed Quantity,Financing Ratio,In Transit Current Value(USD)
0,PTE,651719.00,651719.28,651719.28.7,879568,Ajit Braganza,Anvil Mining Congo SARL,CMR,None,2022-12-01,...,MT,-224854.42,N,N,590.89,671.17,136.00,136.10,1.00,-
1,PTE,651719.00,651719.29,651719.29.3,879568,Ajit Braganza,Anvil Mining Congo SARL,CMR,None,2022-12-01,...,MT,-831920.85,N,N,590.89,671.17,450.00,450.20,1.00,-
2,PTE,661875.00,661875.15,661875.15.72,877159,Keagan Pearson,Khoemacau Copper Mining Pty Ltd,CMR,None,NaT,...,MT,-10.48,N,N,29.07,32.10,0.00,0.00,0.00,10.89
3,PTE,661875.00,661875.15,661875.15.72,877159,Keagan Pearson,Khoemacau Copper Mining Pty Ltd,CMR,None,NaT,...,MT,-10.48,N,N,29.07,32.10,0.00,0.00,0.00,10.89
4,PTE,661875.00,661875.15,661875.15.72,877159,Keagan Pearson,Khoemacau Copper Mining Pty Ltd,CMR,None,NaT,...,MT,-10.48,N,N,29.07,32.10,0.00,0.00,0.00,10.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4658,PTE,675876.00,675876.10,675876.1.1,779374,Vigneshwar Ihm,Newcrest Mining Limited,BL,None,2022-08-24,...,MT,-45050986.84,N,N,10186.24,10937.29,10186.00,10186.24,1.00,-
4659,PTE,650278.00,650278.60,650278.6.1,714302,Ajit Braganza,Newcrest Operations Ltd,BL,None,2022-05-31,...,MT,-46435872.43,N,N,11047.01,11936.42,11047.00,10958.74,1.01,48394315.73
4660,PTE,647544.00,647544.60,647544.6.1,477461,Rodrigo Barrios,Minera Los Pelambres,BL,None,2022-08-24,...,MT,-25285302.54,N,N,10005.60,10906.36,10006.00,10005.60,1.00,-
4661,PTE,673238.00,673238.50,673238.5.3,836486,Nayelli Rosales,Geotechmin OOD,Not Available,None,NaT,...,MT,-6599582.07,N,N,4983.52,5472.22,2804.00,2803.58,1.00,-


In [ ]:
final_BB = pd.concat([bfinal_BB_test,final_BB_test])

In [20]:
#导出结果
df_mia= pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\MIA.xlsx")
result = pd.merge(final_BB_test, df_mia, left_on=["Purchase Assignment Reference"], right_on=["assignment"])
result = result.drop_duplicates(['Purchase Assignment Reference'])
result
result.to_excel(r"C:\Users\Yuyan.Lu\Desktop\\test3.xlsx")

In [ ]:
#search test
search1 = final_BB[(final_BB["PURCHASE ASSIGNMENT"] == "683286.1.1")]
search2 = ffca_qv[(ffca_qv['Purchase Assignment ID'] == '683286.1.1')]
search3 = df_ffca_f1[(df_ffca_f1['Purchase Assignment ID'] == '679100.5.1')]
search4 = df_qv_f1[(df_qv_f1['Assignment ID'] == '679100.5.1')]
search5 = movement_inventory[(movement_inventory["INVENTORY ID"] == '815784')]
search6 = df_inventory_f1[(df_inventory_f1["INVENTORY ID"] == '821629')]
search7 = final_BB_test[(final_BB_test['Purchase Assignment Reference'] == '683286.1.1')]

search6

,INVENTORY ID,COLLATERAL DOCUMENT DATE,PURCHASE ASSIGNMENT,MOVEMENT ID,RECEIPT INCOTERMS,DRY WEIGHT,WET WEIGHT
15833,821629,nan,677992.11.1,nan,nan,nan,nan
